In [1]:
!pip uninstall -y transformers accelerate
!pip install -U "git+https://github.com/huggingface/transformers.git"
!pip install -U accelerate bitsandbytes sentencepiece peft datasets trl


Found existing installation: transformers 5.0.0.dev0
Uninstalling transformers-5.0.0.dev0:
  Successfully uninstalled transformers-5.0.0.dev0
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-wh81n4xd
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-wh81n4xd
  Resolved https://github.com/huggingface/transformers.git to commit e8a6eb3304033fdd9346fe3b3293309fe50de238
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=11350009 sha256=a437367aa18fa4ef10e5c401b92ed9bfe0716677d6ed80b5df0f535cc52dae82
  Stored in directory: /tmp/pip-ephem-wheel-cache-vd42yxkx/wheels/54/cb/3f/83103de5575c534436d6a4686686dead458238dfaf1147e98d
Successfully built transformers
ERROR: pip's dependency resolver does not currently take into acc

In [2]:
import transformers
transformers.__version__


'5.0.0.dev0'

In [3]:
from huggingface_hub import login
login()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "google/gemma-3-4b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# ✅ Fast tokenizer 깨져있어서 무조건 slow tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=False,
    trust_remote_code=True        # ✅ tokenizer도 remote code 필요
)
tokenizer.pad_token = tokenizer.eos_token

# ✅ 핵심!! trust_remote_code=True 넣어야 gemma3 로딩됨
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True        # ✅ 여기 없으면 무조건 KeyError('gemma3') 발생
)


`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [6]:
from datasets import load_dataset

raw_dataset = load_dataset(
    "json",
    data_files="/content/interview_sft_chatml.jsonl"
)["train"]

print(raw_dataset[0])


Generating train split: 0 examples [00:00, ? examples/s]

{'messages': [{'role': 'system', 'content': '당신은 한국어 면접 코치입니다. 답변은 5~8문장으로 간결하고 논리적으로 말하세요. STAR 구조(상황-과제-행동-결과)를 자연스럽게 활용하세요. 메타데이터: {"version": 1.0, "date": 20230116, "occupation": "ICT", "channel": "MOCK", "place": "ONLINE", "gender": "FEMALE", "ageRange": "35-44", "experience": "EXPERIENCED", "question_wordCount": 16, "answer_wordCount": 74, "answer_summary_wordCount": 25, "question_intent": [], "answer_intent": [{"text": "", "expression": "", "category": "attitude"}], "answer_intent_text": NaN, "answer_intent_expression": NaN, "answer_intent_category": "attitude", "question_emotion": [], "answer_emotion": [{"text": "오히려 다양한   지역을 이동하면서 다양한 곳에서 음 새로운 환경을 접하면서 일을 하는 게   그 업무하는 측면에서도 도움이 되는 경우가 더 많았어 가지고   새로운 저는 사람들을 만나고 그 문화를 접하는 게 좋고 재미있습니다.", "expression": "p-interest", "category": "positive"}]}'}, {'role': 'user', 'content': '면접관 질문: 일을 하다 보면 다른 지역에서 업무를 수행하게 될 수도 있는데요 타 지역 근무도 문제가 없으실지 궁금합니다\n좋은 답변을 만들어줘.'}, {'role': 'assistant', 'content': '네 물론 너무 가능합니다. 저는 그리고 사실 좀 다양한 곳에서 이

In [7]:
def messages_to_chatml(messages):
    text = ""
    for m in messages:
        role = m["role"]
        content = m["content"]
        text += f"<start_of_turn>{role}\n{content}<end_of_turn>\n"
    return text


In [8]:
def preprocess(examples):
    texts = [messages_to_chatml(m) for m in examples["messages"]]

    tok = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=512
    )

    tok["token_type_ids"] = [
        [0] * len(ids) for ids in tok["input_ids"]
    ]

    tok["labels"] = tok["input_ids"]
    return tok

dataset = raw_dataset.map(preprocess, batched=True)
dataset = dataset.remove_columns(["messages"])
dataset


Map:   0%|          | 0/727 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'token_type_ids', 'labels'],
    num_rows: 727
})

In [9]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj","k_proj","v_proj","o_proj",
        "gate_proj","up_proj","down_proj"
    ],
)


In [10]:
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()


trainable params: 131,153,920 || all params: 4,431,233,392 || trainable%: 2.9598


In [15]:
class GemmaCollator(DataCollatorForLanguageModeling):
    def __call__(self, features):
        # ✅ 기본 collator로 input_ids, attention_mask, labels 생성
        batch = super().__call__(features)

        # ✅ token_type_ids가 없는 경우 생성해준다
        if "token_type_ids" not in features[0]:
            batch["token_type_ids"] = torch.zeros_like(batch["input_ids"])
        else:
            batch["token_type_ids"] = torch.tensor(
                [f["token_type_ids"] for f in features],
                dtype=torch.long
            )

        return batch

data_collator = GemmaCollator(tokenizer=tokenizer, mlm=False)


In [16]:
from trl import SFTConfig, SFTTrainer

train_config = SFTConfig(
    output_dir="gemma3_lora_output",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,
    optim="paged_adamw_32bit",
)

trainer = SFTTrainer(
    model=model,
    args=train_config,
    train_dataset=dataset,
    data_collator=data_collator,
    peft_config=lora_config,
)


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [17]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.712700
20,1.511100
30,1.362100
40,1.306600
50,1.223500
60,1.236800
70,1.249000
80,1.331100
90,1.223700
100,1.268200


TrainOutput(global_step=364, training_loss=1.2221244311594701, metrics={'train_runtime': 4598.7072, 'train_samples_per_second': 0.158, 'train_steps_per_second': 0.079, 'total_flos': 8386792570798080.0, 'train_loss': 1.2221244311594701, 'entropy': 1.1872801184654236, 'num_tokens': 372224.0, 'mean_token_accuracy': 0.7463086928640094, 'epoch': 1.0})

In [18]:
# ✅ LoRA 어댑터 + tokenizer 저장
trainer.model.save_pretrained("gemma3_lora_interview")
tokenizer.save_pretrained("gemma3_lora_interview")

print("✅ Saved finetuned LoRA adapter & tokenizer")


✅ Saved finetuned LoRA adapter & tokenizer


In [29]:
from huggingface_hub import create_repo

create_repo(
    repo_id="rlawnsrb731/gemma3-finetuned-interview",
    private=False   # ← True로 하면 비공개
)


RepoUrl('https://huggingface.co/rlawnsrb731/gemma3-finetuned-interview', endpoint='https://huggingface.co', repo_type='model', repo_id='rlawnsrb731/gemma3-finetuned-interview')

In [30]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    repo_id="rlawnsrb731/gemma3-finetuned-interview",
    folder_path="gemma3_lora_interview",
    commit_message="Upload finetuned LoRA model"
)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 96.3kB /  525MB            

  ...interview/tokenizer.model: 100%|##########| 4.69MB / 4.69MB            

CommitInfo(commit_url='https://huggingface.co/rlawnsrb731/gemma3-finetuned-interview/commit/7df9e3044f20d735e3f17b7bab18d0f63b64b690', commit_message='Upload finetuned LoRA model', commit_description='', oid='7df9e3044f20d735e3f17b7bab18d0f63b64b690', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rlawnsrb731/gemma3-finetuned-interview', endpoint='https://huggingface.co', repo_type='model', repo_id='rlawnsrb731/gemma3-finetuned-interview'), pr_revision=None, pr_num=None)

In [19]:
from peft import PeftModel

ft_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

ft_model = PeftModel.from_pretrained(
    ft_model,
    "gemma3_lora_interview"
)

print("✅ Finetuned model loaded!")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Finetuned model loaded!


In [24]:
def infer_base(q):
    base_model.eval()  # ✅ inference 모드 강제
    inp = tokenizer(q, return_tensors="pt")

    # ✅ Gemma-3 필수: token_type_ids 강제 생성
    inp["token_type_ids"] = torch.zeros_like(inp["input_ids"])

    inp = inp.to(base_model.device)

    out = base_model.generate(
        **inp,
        max_new_tokens=200
    )

    return tokenizer.decode(out[0], skip_special_tokens=True)


def infer_ft(q):
    ft_model.eval()  # ✅ inference 모드 강제
    inp = tokenizer(q, return_tensors="pt")

    # ✅ token_type_ids 강제 생성
    inp["token_type_ids"] = torch.zeros_like(inp["input_ids"])

    inp = inp.to(ft_model.device)

    out = ft_model.generate(
        **inp,
        max_new_tokens=200
    )

    return tokenizer.decode(out[0], skip_special_tokens=True)


In [25]:
import pandas as pd

eval_df = pd.DataFrame({
    "question": [
        next(m["content"] for m in rec["messages"] if m["role"] == "user")
        for rec in raw_dataset
    ],
    "ground_truth": [
        next(m["content"] for m in rec["messages"] if m["role"] == "assistant")
        for rec in raw_dataset
    ]
})

eval_df.head()


,question,ground_truth
0,면접관 질문: 일을 하다 보면 다른 지역에서 업무를 수행하게 될 수도 있는데요 타 ...,네 물론 너무 가능합니다. 저는 그리고 사실 좀 다양한 곳에서 이동하면서 네 일하는...
1,면접관 질문: 저희 회사에 들어오시게 된다면 가장 가고 싶었던 부서와 그리고 왜 그...,아 저는 가능하다면 어 국가기관에서 네 그 국가 기밀 관련 쪽 부서에서 한 번 일을...
2,면접관 질문: 만약 지원자님께 지금 지원하신 이 직무가 아닌 다른 부서에 배치하게 ...,다른 직무라는 게 음 완전히 다른 직무를 말씀하시는 건 아닐 거잖아요. 제가 무슨 ...
3,면접관 질문: 이번에는 본인이 창의적으로 문제를 해결한 경험이 어떤 게 있는지 과정...,음 아 제가 초등학생 때 어 꽤 어렸을 땐데 이게 뭐 딱히 문제라고 까지는 아니지만...
4,면접관 질문: 지원자분께서는 최근에 정보 보안에 관련하여 알고 있는 이슈가 있으신가...,어 이슈라면 음 그냥 지금 바로 확 생각이 난 건 네 지금 최근 계속 그대로 말 그...


In [26]:
from tqdm import tqdm

def build_ragas_df(df, infer_fn):
    data = {"question":[], "answer":[], "ground_truth":[], "contexts":[]}

    for i in tqdm(range(len(df))):
        q  = df.iloc[i]["question"]
        gt = df.iloc[i]["ground_truth"]

        pred = infer_fn(q)

        data["question"].append(q)
        data["answer"].append(pred)
        data["ground_truth"].append(gt)
        data["contexts"].append([])  # 문서 기반 RAG 아니라 빈 리스트 유지

    return pd.DataFrame(data)


In [27]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy

df_base = build_ragas_df(eval_df, infer_base)
df_ft   = build_ragas_df(eval_df, infer_ft)

base_scores = evaluate(df_base, metrics=[faithfulness(), answer_relevancy()])
ft_scores   = evaluate(df_ft, metrics=[faithfulness(), answer_relevancy()])

print("✅ Base model RAGAS:", base_scores)
print("✅ Finetuned model RAGAS:", ft_scores)



  1%|          | 5/727 [01:23<3:19:50, 16.61s/it]


KeyboardInterrupt: 

In [ ]:
compare = pd.DataFrame({
    "metric": base_scores.keys(),
    "base": base_scores.values(),
    "finetuned": ft_scores.values(),
})

compare["delta"] = compare["finetuned"] - compare["base"]
compare
